In [2]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint

def run_NASEM():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    user_diet, animal_input, equation_selection = nd.read_csv_input('./input.csv')
        
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')
    feed_data.reset_index(inplace=True)
    feed_data = feed_data.rename(columns={"Fd_Name": "Feedstuff"})

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()
    diet_info_initial.insert(1, 'Fd_DMInp', Fd_DMInp.reindex(diet_info_initial['Feedstuff']).values)
    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    diet_info_initial['Fd_DNDF48'] = 0  

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del(list_of_feeds, Fd_DMInp)

########################################
# Step 2: DMI Equations
########################################
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8: 
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'], 
            animal_input['Trg_MilkProd'], 
            animal_input['An_BW'], 
            animal_input['An_BCS'],
            animal_input['An_LactDay'], 
            animal_input['Trg_MilkFatp'], 
            animal_input['Trg_MilkTPp'], 
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers    
    elif equation_selection['DMIn_eqn'] in [2,3,4,5,6,7,12,13,14,15,16,17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'], 
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF, 
            animal_input['An_BW'], 
            animal_input['An_BW_mature'], 
            animal_input['An_PrePartWk'], 
            nd.coeff_dict)

    
    elif equation_selection['DMIn_eqn'] in [10,11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'], 
            animal_input['An_BW'], 
            animal_input['An_PrePartWk'], 
            animal_input['An_GestDay'], 
            animal_input['An_GestLength'], 
            Dt_NDF, 
            nd.coeff_dict)
        
    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: "+ str(equation_selection['DMIn_eqn']) )

    del(Dt_NDF)

########################################
# Step 3: Feed Based Calculations
########################################
    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=nd.coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions
    
    diet_data = nd.calculate_diet_data(diet_info, 
                                       animal_input['DMI'], 
                                       animal_input['An_BW'],
                                       nd.coeff_dict)
    # diet_data contains everything starting with "Dt_"

    return animal_input, diet_info, equation_selection, diet_data

### RUN MODEL ###
animal_input, diet_info, equation_selection, diet_data = run_NASEM()

# pprint(diet_data)

{'Dt_ADF': 22.94867121257886,
 'Dt_ADFIn': 5.627243668036462,
 'Dt_ADF_NDF': 0.6945789612618369,
 'Dt_AFIn': 37.7598260298646,
 'Dt_Ash': 7.188026615138312,
 'Dt_AshIn': 1.7625760062980653,
 'Dt_B_Carotene': 0.0,
 'Dt_B_CaroteneIn': 0.0,
 'Dt_Biotin': 0.0,
 'Dt_BiotinIn': 0.0,
 'Dt_C120': 0.007894794002831808,
 'Dt_C120In': 0.0019358824374343874,
 'Dt_C120_FA': 0.33574235641259936,
 'Dt_C140': 0.020801153856342573,
 'Dt_C140In': 0.005100650937113762,
 'Dt_C140_FA': 0.8846118605912242,
 'Dt_C160': 0.3481389689174488,
 'Dt_C160In': 0.08536715656824762,
 'Dt_C160_FA': 14.805325856693804,
 'Dt_C161': 0.01807261545976891,
 'Dt_C161In': 0.004431586036889934,
 'Dt_C161_FA': 0.768575152033749,
 'Dt_C180': 0.05805379460916293,
 'Dt_C180In': 0.014235370976112842,
 'Dt_C180_FA': 2.4688570460207186,
 'Dt_C181c': 0.5732967504724736,
 'Dt_C181cIn': 0.14057809618335526,
 'Dt_C181c_FA': 24.380623719665525,
 'Dt_C181t': 0.004252096150796392,
 'Dt_C181tIn': 0.0010426564971367834,
 'Dt_C181t_FA': 0.18082

C:\Users\Braed\OneDrive - University of Guelph\NASEM-Model-Python\src\nasem_dairy\NASEM_equations\dev_nutrient_intakes.py:807: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_diet_info[f"{column_name}In"] = complete_diet_info.apply(lambda row: row['Fd_DMIn'] * row[f"{column_name}"], axis=1)
C:\Users\Braed\OneDrive - University of Guelph\NASEM-Model-Python\src\nasem_dairy\NASEM_equations\dev_nutrient_intakes.py:807: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_diet_info[f"{column_name}In"] = complete_diet_info.ap

In [19]:
# Refactor AA_calculations
import nasem_dairy as nd
import numpy as np

# Drop get diet data section
# Isolate calculations in Define Variables
# Put the calculations on Diet Data into calculate_diet_info
# Keep calculations on AA values as is, in new function

### Need to include check_coeffs_in_coeff_dict import in .py

def calculate_f_mPrt_max(An_305RHA_MlkTP, coeff_dict):
    req_coeffs = ['K_305RHA_MlkTP']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    f_mPrt_max = 1 + coeff_dict['K_305RHA_MlkTP'] * (An_305RHA_MlkTP / 280 - 1)       # Line 2116, 280kg RHA ~ 930 g mlk NP/d herd average
    return f_mPrt_max

def calculate_Du_MiCP_g(Du_MiN_g):
    Du_MiCP_g = Du_MiN_g * 6.25     # Line 1163
    return Du_MiCP_g

def calculate_Du_MiTP_g(Du_MiCP_g, coeff_dict):
    req_coeffs = ['fMiTP_MiCP']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    Du_MiTP_g = coeff_dict['fMiTP_MiCP'] * Du_MiCP_g     # Line 1166
    return Du_MiTP_g

# Amino acids will get their own dev.py file for functions
# Will keep all individual AA calulations in one dataframe

def calculate_Du_AAMic(Du_MiTP_g, AA_list, coeff_dict):
    req_coeffs = ['MiTPArgProf', 'MiTPHisProf','MiTPIleProf', 'MiTPLeuProf',
                  'MiTPLysProf', 'MiTPMetProf', 'MiTPPheProf','MiTPThrProf', 
                  'MiTPTrpProf', 'MiTPValProf']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    AA_coeffs = np.array([coeff_dict[f"MiTP{AA}Prof"] for AA in AA_list])
    Du_AAMic = Du_MiTP_g * AA_coeffs / 100   # Line 1573-1582
    return Du_AAMic

def calculate_Du_IdAAMic(Du_AAMic, coeff_dict):
    req_coeffs = ['SI_dcMiCP']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)    
    Du_IdAAMic = Du_AAMic * coeff_dict['SI_dcMiCP'] / 100
    return Du_IdAAMic

def calculate_Abs_AA_g(diet_data, Du_IdAAMic, AA_list):
    AA_coeffs = np.array([diet_data[f"Dt_Id{AA}_RUPIn"] for AA in AA_list])
    Abs_AA_g = Du_IdAAMic + AA_coeffs
    return Abs_AA_g

def calculate_mPrtmx_AA(AA_list, coeff_dict):
    req_coeffs = ['mPrt_k_Arg_src', 'mPrt_k_His_src', 'mPrt_k_Ile_src', 'mPrt_k_Leu_src', 
                  'mPrt_k_Lys_src', 'mPrt_k_Met_src', 'mPrt_k_Phe_src', 'mPrt_k_Thr_src', 
                  'mPrt_k_Trp_src', 'mPrt_k_Val_src', 'mPrt_k_EAA2_src']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    AA_coeffs = np.array([coeff_dict[f'mPrt_k_{AA}_src'] for AA in AA_list])
    mPrtmx_AA = -(AA_coeffs**2) / (4 * coeff_dict['mPrt_k_EAA2_src'])
    return mPrtmx_AA

def calculate_mPrtmx_AA2(mPrtmx_AA, f_mPrt_max):
    mPrtmx_AA2 = mPrtmx_AA * f_mPrt_max     # Line 2149-2158
    return mPrtmx_AA2

def calculate_AA_mPrtmx(AA_list, coeff_dict):
    req_coeffs = ['mPrt_k_Arg_src', 'mPrt_k_His_src', 'mPrt_k_Ile_src', 'mPrt_k_Leu_src', 
                  'mPrt_k_Lys_src', 'mPrt_k_Met_src', 'mPrt_k_Phe_src', 'mPrt_k_Thr_src', 
                  'mPrt_k_Trp_src', 'mPrt_k_Val_src', 'mPrt_k_EAA2_src']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    AA_coeffs = np.array([coeff_dict[f"mPrt_k_{AA}_src"] for AA in AA_list])
    AA_mPrtmx = -AA_coeffs / (2 * coeff_dict['mPrt_k_EAA2_src'])
    return AA_mPrtmx

def calculate_mPrt_AA_01(AA_mPrtmx, AA_list, coeff_dict):
    req_coeffs = ['mPrt_k_Arg_src', 'mPrt_k_His_src', 'mPrt_k_Ile_src', 'mPrt_k_Leu_src', 
                  'mPrt_k_Lys_src', 'mPrt_k_Met_src', 'mPrt_k_Phe_src', 'mPrt_k_Thr_src', 
                  'mPrt_k_Trp_src', 'mPrt_k_Val_src', 'mPrt_k_EAA2_src']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)
    AA_coeffs = np.array([coeff_dict[f"mPrt_k_{AA}_src"] for AA in AA_list])
    mPrt_AA_01 = AA_mPrtmx * 0.1 * AA_coeffs + (AA_mPrtmx * 0.1)**2 * coeff_dict['mPrt_k_EAA2_src']
    return mPrt_AA_01

def calculate_mPrt_k_AA(mPrtmx_AA2, mPrt_AA_01, AA_mPrtmx):
    condition = (mPrtmx_AA2**2 - mPrt_AA_01 * mPrtmx_AA2 <= 0) | (AA_mPrtmx == 0)
    # Check for sqrt of 0 or divide by 0 errors and set value to 0 if encountered
    mPrt_k_AA = np.where(condition, 
                         0,
                         -(2 * np.sqrt(mPrtmx_AA2**2 - mPrt_AA_01 * mPrtmx_AA2) - 
                           2 * mPrtmx_AA2) / (AA_mPrtmx * 0.1)
                         )
    return mPrt_k_AA


In [20]:
# Test new functions

f_mPrt_max = calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'], nd.coeff_dict)
Du_MiCP_g = calculate_Du_MiCP_g(328.384834667338)
Du_MiTP_g = calculate_Du_MiTP_g(Du_MiCP_g, nd.coeff_dict)

AA_list = ['Arg', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
AA_values = pd.DataFrame(index=AA_list)

AA_values['Du_AAMic'] = calculate_Du_AAMic(Du_MiTP_g, AA_list, nd.coeff_dict)
AA_values['Du_IdAAMic'] = calculate_Du_IdAAMic(AA_values['Du_AAMic'], nd.coeff_dict)
AA_values['Abs_AA_g'] = calculate_Abs_AA_g(diet_data, AA_values['Du_IdAAMic'], AA_list)
AA_values['mPrtmx_AA'] = calculate_mPrtmx_AA(AA_list, nd.coeff_dict)
AA_values['mPrtmx_AA2'] = calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'], f_mPrt_max)
AA_values['AA_mPrtmx'] = calculate_AA_mPrtmx(AA_list, nd.coeff_dict)
AA_values['mPrt_AA_01'] = calculate_mPrt_AA_01(AA_values['AA_mPrtmx'], AA_list, nd.coeff_dict)
AA_values['mPrt_k_AA'] = calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'], AA_values['mPrt_AA_01'], AA_values['AA_mPrtmx'])


In [14]:
# Refactor calculate_Du_MiN_NRC2021_g

def calculate_Rum_dcNDF(Dt_NDFIn, Dt_DMIn, Dt_StIn, Dt_CPIn, Dt_ADFIn, Dt_ForWet):
    Rum_dcNDF = -31.9 + 0.721 * Dt_NDFIn / Dt_DMIn * 100 - \
            0.247 * Dt_StIn / Dt_DMIn * 100 + \
            6.63 * Dt_CPIn / Dt_DMIn * 100 - \
            0.211 * (Dt_CPIn / Dt_DMIn * 100) ** 2 - \
            0.387 * Dt_ADFIn / Dt_DMIn / (Dt_NDFIn / Dt_DMIn) * 100 - \
            0.121 * Dt_ForWet + 1.51 * Dt_DMIn

    if Rum_dcNDF < 0.1 or Rum_dcNDF is None:                                                # Line 984
        Rum_dcNDF = 0.1
    return Rum_dcNDF

def calculate_Rum_DigNDFIn(Rum_dcNDF, Dt_NDFIn):
    Rum_DigNDFIn = Rum_dcNDF / 100 * Dt_NDFIn
    return Rum_DigNDFIn

def calculate_Rum_dcSt(Dt_DMIn, Dt_ForNDF, Dt_StIn, Dt_ForWet):
    Rum_dcSt = 70.6 - 1.45*(Dt_DMIn) + 0.424*Dt_ForNDF + \
            1.39*(Dt_StIn)/(Dt_DMIn)*100 - \
            0.0219*((Dt_StIn)/(Dt_DMIn)*100)**2 - \
            0.154*Dt_ForWet

    if Rum_dcSt < 0.1:                                                                      # Line 992
        Rum_dcSt = 0.1            
    elif Rum_dcSt > 100:                                                                    # Line 993
        Rum_dcSt = 100 
    return Rum_dcSt

def calculate_Rum_DigStIn(Rum_dcSt, Dt_StIn):
    Rum_DigStIn = Rum_dcSt / 100 * Dt_StIn                                                   # Line 998
    return Rum_DigStIn

def calculate_Du_MiN_NRC2021_g(MiN_Vm, Rum_DigNDFIn, Rum_DigStIn, An_RDPIN_g, coeff_dict):
    req_coeffs = ['KmMiNRDNDF', 'KmMiNRDSt']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

    Du_MiN_NRC2021_g = MiN_Vm / (1 + coeff_dict['KmMiNRDNDF'] / Rum_DigNDFIn + coeff_dict['KmMiNRDSt'] / Rum_DigStIn)   # Line 1126

    if Du_MiN_NRC2021_g > 1 * An_RDPIN_g/6.25:  # Line 1130
        Du_MiN_NRC2021_g = 1 * An_RDPIN_g/6.25
    else:
        Du_MiN_NRC2021_g = Du_MiN_NRC2021_g

    return Du_MiN_NRC2021_g

def calculate_Du_MiN_VTln_g(Dt_rOMIn, Dt_ForNDFIn, Rum_DigStIn, Rum_DigNDFIn, coeff_dict):
    req_coeffs = ['Int_MiN_VT',
                  'KrdSt_MiN_VT', 
                  'KrdNDF_MiN_VT', 
                  'KRDP_MiN_VT', 
                  'KrOM_MiN_VT', 
                  'KForNDF_MiN_VT', 
                  'KrOM2_MiN_VT', 
                  'KrdStxrOM_MiN_VT', 
                  'KrdNDFxForNDF_MiN_VT'
                  ]
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

    # Line 1144-1146
    Du_MiN_VTln_g = coeff_dict['Int_MiN_VT'] + coeff_dict['KrdSt_MiN_VT'] * Rum_DigStIn + coeff_dict['KrdNDF_MiN_VT'] * Rum_DigNDFIn 
    + coeff_dict['KRDP_MiN_VT'] * An_RDPIn + coeff_dict['KrOM_MiN_VT'] * Dt_rOMIn + coeff_dict['KForNDF_MiN_VT'] * Dt_ForNDFIn 
    + coeff_dict['KrOM2_MiN_VT'] * Dt_rOMIn ** 2 
    + coeff_dict['KrdStxrOM_MiN_VT'] * Rum_DigStIn * Dt_rOMIn + coeff_dict['KrdNDFxForNDF_MiN_VT'] * Rum_DigNDFIn * Dt_ForNDFIn

    return Du_MiN_VTln_g

def calculate_Du_MiN_VTnln_g(An_RDPIn, Rum_DigNDFIn, Rum_DigStIn):
    Du_MiN_VTnln_g = 7.47 + 0.574 * An_RDPIn * 1000 / (1 + 3.60 / Rum_DigNDFIn + 12.3 / Rum_DigStIn)    # Line 1147
    return Du_MiN_VTnln_g



# Refactor calculate_Du_MiN_g
# Need and equation selection variable, Don't need a function for Du_MiN_g
# There are 3 different equations to choose from, include equation selection here
    # MiN_eqn is the variable name

# if eqn_select == 1:
#     Du_MiN_g = calculate_Du_MiN_NRC2021_g(MiN_Vm, Rum_DigNDFIn, Rum_DigStIn, An_RDPIN_g, nd.coeff_dict)        
# elif eqn_select == 2:
#     Du_MiN_g = calculate_Du_MiN_VTln_g(Dt_rOMIn, Dt_ForNDFIn, Rum_DigStIn, Rum_DigNDFIn, nd.coeff_dict)
# elif eqn_select ==3:
#     Du_MiN_g = calculate_Du_MiN_VTnln_g(An_RDPIn, Rum_DigNDFIn, Rum_DigStIn)
# else:
#     print("Invalid equation number")
#     # Should raise error here



In [ ]:
from icecream import ic

# Test new functions
An_RDPIn = 1   # Set An_RDPIn
An_RDPIN_g = An_RDPIn * 1000
    # There are a lot of these unit coversions to g, should try and do them all at once if possible
Dt_NDFIn = 8.10166182432373
Dt_DMIn = 24.521
Dt_StIn = 4.96654799368879
Dt_CPIn = 5.16656423020988
Dt_ADFIn = 5.62724383705589
Dt_ForWet = 22.3214307767857
Dt_ForNDF = 23.4844869990251

Rum_dcNDF = ic(calculate_Rum_dcNDF(Dt_NDFIn, Dt_DMIn, Dt_StIn, Dt_CPIn, Dt_ADFIn, Dt_ForWet))# Start by testing all functions with values from R

Rum_DigNDFIn = ic(calculate_Rum_DigNDFIn(Rum_dcNDF, Dt_NDFIn))

Rum_dcSt = ic(calculate_Rum_dcSt(Dt_DMIn, Dt_ForNDF, Dt_StIn, Dt_ForWet))

Rum_DigStIn = ic(calculate_Rum_DigStIn(Rum_dcSt, Dt_StIn))




